# Dependencies 

In [ ]:
%load_ext autoreload
%autoreload 2

# For debugging turn on logging to console
import logging
logging.basicConfig(level=logging.DEBUG)
log = logging.getLogger(__name__)

# mongodb
import pymongo as pm

# fine-tuned newspaper lib
from resources.newspaper import newspaper
from resources.newspaper.newspaper.source import Source
from resources.newspaper.newspaper.article import Article

import bs4 as bs
from urllib.parse import urljoin
from dateutil.parser import parse as date_parser
from time import sleep
import random
import pytz
import datetime

import requests

In [ ]:
# for infinite scroll page
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import sys

# import unittest, time, re

# to divert selenium log stream away
logging.getLogger('selenium').setLevel(logging.WARNING)

# Helpers

In [1]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn, maxPoolSize=200)

# define db 
# DB_NAME = 'scrape'
DB_NAME = 'FINALP'
db = client[DB_NAME]

def saveToDB(db, collection, url, html, meta={}):
    """
    Saves a document to mongoDB, making sure there are no duplicates by 
    'url' value
    
    Parameters:
    --------
    db, collection  : mongo db connection
    url, html, meta : values to store
    
    Returns:
    --------
    Saved document
    """
    collection = db[collection]
    collection.update_one(
        {'url' : url},
        {
            '$set':
                {'url' : url,
                 'html' : html,
                 'meta' : meta
                }
        }
        ,
        upsert=True
    )
    log.debug(f'Saved to DB')

def scrape(url, db, collection):
    '''
    Scrapes an article from the 'url' up to the 'latest_date'
    
    Parameters:
    --------
    url         : main news website url
    date        : YYYY-MM-DD
    db          : database name
    collection  : mongodb collection
    
    Returns:
    --------
    Article's html and features stored to db, 
    Article's publish date
    
    '''
    log.debug(f"Exctracting features from {url}")
    try:
        article = Article(url)
        article.download()
        # the below method may only extract a snippet... 
        # check the database for results of text extraction
        # and apply additional processing if needed after 
        # article has been stored in the DB
        # see code below Newrepublic for example
        article.parse()
    except Exception as e:
        log.critical(f'Data not saved: {e}')
        return datetime.datetime.now()
    
    saveToDB(db, collection, article.url, article.html, meta={
        'date'    :article.publish_date,
        'title'   :article.title,
        'text'    :article.text,
        'authors' :article.authors
    })
    
    return article.publish_date

def addToDB(DB_NAME,COL_NAME,PATH,FILE):
    '''
    Imports a file into mongoDB
    
    Parameters:
    --------
    DB_NAME : Name of the database to connect to
    COL_NAME: Name of the collection to create
    PATH    : Path to folder with the file
    FILE  : Filename
    
    Returns:
    --------
    Collection COL_NAME in DB_NAME database
    '''
    !mongoimport --db {DB_NAME} --collection {COL_NAME} --file {PATH+FILE} --batchSize 1
    print(f'Collection {COL_NAME} in {DB_NAME} database created')

In [ ]:
from bson.objectid import ObjectId

def show_doc(db, collection, id):
    '''
    Finds a document by 'id' and prints contents to the console
    
    Parameters:
    --------
    db         : database name
    collection : mongodb collection
    id         : mongodb document id
    
    Returns:
    --------
    Prints first 100 symbols of each document's key to console
    '''
    
    doc = db[collection].find_one({'_id':ObjectId(id)})
    for k in doc:
        print(f"{k} : {str(doc[k])}")

In [ ]:
# powered by NewsAPI.org
SOURCES = {
    'left'  : [
        'https://newrepublic.com',
        'https://www.motherjones.com'
# 3. Slate
# 4. The Intercept
# 5. Daily Beast
# 6. The Atlantic
# 7. Washington Post
# 8. Politico
# 9. The Guardian
# 10. BBC
    ],
    'right' : [
        'https://www.breitbart.com'
# 2. Fox News
# 3. New York Post
# 4. The American Conservative
# 5. Washington Times
# 6. Daily Wire
# 7. The Fiscal Times
# 8. The Hill
# 9. The Daily Caller
# 10. Reason
    ]
}

# Some comands to keep dbs clean

In [ ]:
# deletes all 'meta' fields from all docs
# htmlCol.update({}, {$unset: {meta:1}}, false, true); # mongo shell comand
htmlCol.update({}, {'$unset': {'meta':1}}, multi=True) # pymongo way

In [ ]:
# leaves only unique documents by 'url' field

htmlCol.create_index(
    "url",
    unique=True
)

In [ ]:
# pymongo 'find' returns cursor that allows iterating through results
# calling first object [0] allows accessing the dictionary with results
# the ['html'] is the key in the dictionary
html = htmlCol.find({'url':'http://www.msnbc.com/velshi-ruhle/watch/jeff-sessions-is-justifying-harsh-immigration-policy-with-the-bible-1256689731629'},\
            projection={'html':True, '_id':False})[0]['html']

In [ ]:
# find documents NOT containing a 'tag': regex expression
import re
tag = re.compile('dek___3AQpw.')
docs = htmlCol.find({"html" : {'$not': tag}})
for d in docs[:20]: print(d['url'])

# Multithreading

In [2]:
from multiprocessing import Process

# use multiprocessing to extract features
def func():
    DB_NAME = 'scrape'
    db = pm.MongoClient(host='localhost', port=27017, maxPoolSize=500)[DB_NAME]

    for collection in ['left','right']: docs_parser(db[collection])

proc = Process(target=func)
proc.start()

Process Process-1:
Traceback (most recent call last):
  File "/Users/yegor3/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/yegor3/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-fd76abbece9f>", line 6, in func
    db = pm.MongoClient(host='localhost', port=27017, maxPoolSize=500)[DB_NAME]
NameError: name 'pm' is not defined


# Newrepublic

In [ ]:
# name of collection for this media
collection = 'newRep'
source = 'https://newrepublic.com/latest'
page   = 91

earliest_date = date_parser('2017-01-01')

while True:
    log.debug(f'PROCESSING PAGE: {page}')
    s = Source(source+'?page='+str(page))
    s.download()

    soup = bs.BeautifulSoup(s.html,'lxml')

    # line below needs to be updated per news source
    # to include the specific tags for article text 
    # defined differently for each site
    for section in soup.findAll('article'):
        url = urljoin(s.url, section.a['href'])
        log.debug(f'Processing url: {url}')
        
        article_date = scrape(url, db, collection)

    if article_date < earliest_date:
        log.debug(f'Reached earliest date requested: {article_date}')
        break
    page += 1

In [ ]:
# update text field to include more data
for doc in db[collection].find():
    soup = bs.BeautifulSoup(doc['html'],'lxml')
    text = ''
    for div in soup.findAll('div',{"class": "content-body"}):
        text += div.text
    if len(doc['meta']['text']) < len(text):
        db[collection].update_one(
            {'url' : doc['url']},
            {
                '$set':
                    {
                     'meta.text' : text
                    }
            }
            ,
            upsert=True
        )

In [ ]:
url = 'https://newrepublic.com/article/139550/legacy-altamont'
doc = db[collection].find_one({'url':url})
doc['meta']['text']

# The Atlantic

In [ ]:
utc=pytz.UTC

# name of collection for this media
collection = 'theAtlantic'
source = 'https://www.theatlantic.com/latest/'

# start 'page' at '1' but if you run across an error
# efficient way is to update this page to the same number
# where you experienced the error AFTER you correct the error in the code
# then rerun this cell
page   = 175

earliest_date = utc.localize(date_parser('2017-01-01'))

while True:
    log.debug(f'\n\n PROCESSING PAGE: {source+"?page="+str(page)}\n\n\
              ====================================\n\n')
    s = Source(source+'?page='+str(page))
    page += 1
    s.download()
    soup = bs.BeautifulSoup(s.html,'lxml')

    # line below needs to be updated per news source
    # to include the specific tags for article text 
    # defined differently for each site
    for section in soup.findAll('li', {"class":"article"}):
        url = urljoin(s.url, section.a['href'])
        log.debug(f'Processing url: {url}')
        
        try:
            article_date = scrape(url, db, collection)
        except Exception:
            article_date = earliest_date + 1 #to make sure scraping continues
        
        # the Atlantic blocks right away after few quick downloads
        # so it requires sleeping, testing showed 1 to 5 seconds is enough
#         sleep(random.uniform(1,5))

    try:
        if article_date < earliest_date:
            log.debug(f'Reached earliest date requested: {article_date}')
            break
    except Exception as e:
        log.debug(f'Exception: {e}')
        continue
    

# Politico

In [ ]:
# setup
# use your newsapi key
# update collection, date and source
collection = 'Politoco'
page = 1
earliest_date = '2017-01-01'
latest_date = '2017-09-13'
source = 'politico'
pageSize = 100
apiKey = 'fd1386b678fd4524b2aa84e3bee1f8f7'

params = {
    'apiKey' : apiKey,
    'pageSize' : pageSize,
    'page' : page,
    'from' : earliest_date,
    'to'   : latest_date,
    'sources': source
}

# base url
api_url = 'https://newsapi.org/v2/everything?'

# get first request to obtain total news count
r = requests.get(api_url, params=params)
totalPages = r.json()['totalResults']//100
log.debug(f'Total pages ({pageSize} per page) for {source} results from {earliest_date} to {latest_date} is {totalPages}')

# scrape news
for p in range(page,totalPages+1):
    log.debug(f'\n\n PROCESSING PAGE: {source+"?page="+str(page)}\n\n\
              ====================================\n\n')
    
    params = {
        'apiKey' : apiKey,
        'pageSize' : pageSize,
        'page' : page,
        'from' : earliest_date,
        'to'   : latest_date,
        'sources': source
    }
    
    r = requests.get(api_url, params=params)

    page += 1
    
    if r.json()['articles']:
        for a in r.json()['articles']:
            try:
                log.debug(f"Processing url: {a['url']}")
                article_date = scrape(a['url'], db, collection)
            except Exception:
                article_date = earliest_date + 1
    else:
        log.debug(f"Response doesn't have articles. Response: {r.json()}")
        

In [ ]:
r.json()['totalResults']

# Washington Times

In [ ]:
# setup
# use your newsapi key
# update collection, date and source
collection = 'WashTimes'

page = 1
earliest_date = '2017-01-01'
latest_date = '2017-12-31'
source = 'the-washington-times'
pageSize = 100
apiKey = 'fd1386b678fd4524b2aa84e3bee1f8f7'

params = {
    'apiKey'   : apiKey,
    'pageSize' : pageSize,
    'page'     : page,
    'from'     : earliest_date,
    'to'       : latest_date,
    'sources'  : source
}

# base url
api_url = 'https://newsapi.org/v2/everything?'

# get first request to obtain total news count
r = requests.get(api_url, params=params)
totalPages = r.json()['totalResults']//100

log.debug(f'TOTAL PAGES FOR {source}: {totalPages}')

# scrape news
for p in range(page,totalPages):
    log.debug(f'\n\n PROCESSING PAGE: {source+"?page="+str(page)}\n\n\
              ====================================\n\n')
    
    params = {
        'apiKey'   : apiKey,
        'pageSize' : pageSize,
        'page'     : page,
        'from'     : earliest_date,
        'to'       : latest_date,
        'sources'  : source
    }
    
    r = requests.get(api_url, params=params)

    page += 1
    
    for a in r.json()['articles']:
        try:
            log.debug(f"Processing url: {a['url']}")
            if db[collection].find({'url':a['url']}):
                log.debug(f'URL exists in DB. Skipping.')
            else:
                article_date = scrape(a['url'], db, collection)
        except Exception:
            article_date = earliest_date + 1

In [ ]:
r.json()

# Slate

In [ ]:
collection = 'slate'
source = 'https://slate.com/news-and-politics/'
page   = 118

utc=pytz.UTC
earliest_date = utc.localize(date_parser('2017-01-01'))

while True:
    log.debug(f'\n\n PROCESSING PAGE: {source+str(page)}\n\n\
              ====================================\n\n')
    s = Source(source+str(page))
    page += 1
    s.download()
    soup = bs.BeautifulSoup(s.html,'lxml')

    # line below needs to be updated per news source
    # to include the specific tags for article text 
    # defined differently for each site
    for link in soup.find('div', {"class":"topic-stories-list"}).findChildren("a" , recursive=False):
        url = link['href']
        log.debug(f'Processing url: {url}')
        
        try:
            article_date = scrape(url, db, collection)
        except Exception:
            article_date = earliest_date + 1 #to make sure scraping continues
        
        # the Atlantic blocks right away after few quick downloads
        # so it requires sleeping, testing showed 1 to 5 seconds is enough
#         sleep(random.uniform(1,5))

    try:
        if article_date < earliest_date:
            log.debug(f'Reached earliest date requested: {article_date}')
            break
    except Exception as e:
        log.debug(f'Exception: {e}')
        continue

# The Intercept

In [ ]:
collection = 'theintercept'
source = 'https://theintercept.com'

logging.getLogger('selenium').setLevel(logging.WARNING)

class Sel(unittest.TestCase):
    def setUp(self):
        self.driver = webdriver.Chrome()
        self.driver.implicitly_wait(30)
        self.base_url = source
        self.verificationErrors = []
        self.accept_next_alert = True
    def getPage(self):
        driver = self.driver
        delay = 2
        driver.get(self.base_url)
        html_source = driver.page_source
        self.html = html_source.encode('utf-8')
        return self.html
    def scrollDown(self):
        driver = self.driver
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        self.html = driver.page_source.encode('utf-8')
        return driver.page_source 
    def shutdown(self):
        driver = self.driver
        driver.quit()

data = Sel()

data.setUp()
data.getPage()
page = 1

utc=pytz.UTC
earliest_date = utc.localize(date_parser('2017-01-01'))

scraped_urls = []

while True:
    log.debug(f'NEXT SCROLL (#{page})')
    page += 1
    
    html = data.scrollDown()
    soup = bs.BeautifulSoup(html,'lxml')

    for link in soup.find('div', {"id":"recently"}).findAll('a'):
        url = urljoin(source, link['href'])
        if url and url not in scraped_urls:
            scraped_urls.append(url)
            log.debug(f'Processing url: {url}')
            article_date = scrape(url, db, collection)

    try:
        if article_date < earliest_date:
            log.debug(f'Reached earliest date requested: {article_date}')
            break
    except Exception as e:
        log.debug(f"Something is wrong: {e}")
data.shutdown()

# The daily beast

In [12]:
collection = 'thedailybeast'
source = 'https://www.thedailybeast.com/sitemap'
earliest_date = date_parser('2017-01-01')
s = Source(source)
s.download()
soup = bs.BeautifulSoup(s.html,'lxml')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /sitemap HTTP/1.1" 200 None


In [21]:
months = ['https://www.thedailybeast.com/sitemap/2018/1/article',
 'https://www.thedailybeast.com/sitemap/2018/2/article',
 'https://www.thedailybeast.com/sitemap/2018/3/article',
 'https://www.thedailybeast.com/sitemap/2018/4/article',
 'https://www.thedailybeast.com/sitemap/2018/5/article',
 'https://www.thedailybeast.com/sitemap/2018/6/article',
 'https://www.thedailybeast.com/sitemap/2018/7/article',
 'https://www.thedailybeast.com/sitemap/2017/1/article',
 'https://www.thedailybeast.com/sitemap/2017/2/article',
 'https://www.thedailybeast.com/sitemap/2017/3/article',
 'https://www.thedailybeast.com/sitemap/2017/4/article',
 'https://www.thedailybeast.com/sitemap/2017/5/article',
 'https://www.thedailybeast.com/sitemap/2017/6/article',
 'https://www.thedailybeast.com/sitemap/2017/7/article',
 'https://www.thedailybeast.com/sitemap/2017/8/article',
 'https://www.thedailybeast.com/sitemap/2017/9/article',
 'https://www.thedailybeast.com/sitemap/2017/10/article',
 'https://www.thedailybeast.com/sitemap/2017/11/article',
 'https://www.thedailybeast.com/sitemap/2017/12/article']

In [ ]:
for month in months:
    log.debug(f"\n=======PROCESSING PAGE: {month}\n")
    s = Source(month)
    s.download()
    soup = bs.BeautifulSoup(s.html,'lxml')
    for a in soup.find('div',{'class':'SitemapMonthPage__articles-wrapper'}).findAll('a'):
        url = urljoin(source, a['href'])
        log.debug(f'Processing url: {url}')
        
        article_date = scrape(url, db, collection) 

# The Guardian

In [8]:
collection    = 'theguardian'
source        = 'https://content.guardianapis.com/search'
apikey        = '53aee7b4-8c94-495b-b13b-ea7214ba0ca2'
earliest_date = '2017-01-01'
latest_date   = '2017-02-26'
section       = 'us-news'
edition       = 'us'
page          = 1
page_size     = 200

params = {
    'api-key'   : apikey,
    'edition'   : edition,
    'section'   : section,
    'from-date' : earliest_date,
    'to-date'   : latest_date,
    'page-size' : page_size,
    'page'      : page
}

r = requests.get(source, params=params)

response = r.json()['response']
pages    = response['pages']

for page in range(1,pages):
    log.debug(f"\n================================\nPROCESSING PAGE: {page}\n")
    
    params = {
    'api-key'   : apikey,
    'edition'   : edition,
    'section'   : section,
    'from-date' : earliest_date,
    'to-date'   : latest_date,
    'page-size' : page_size,
    'page'      : page
    }
    
    r = requests.get(source, params=params)
    response = r.json()['response']
    
    for article in response['results']:
        apiUrl = article['apiUrl']
        log.debug(f'Processing apiUrl: {apiUrl}')
        a = requests.get(apiUrl, params={'api-key':apikey,'show-fields': 'all'}).json()['response']['content']

        html  = a['fields']['main'] + a['fields']['body']
        url   = a['webUrl']
        log.debug(f'Weburl: {url}')
        
        try:
            saveToDB(db, collection, url, html, meta={
                'date'    : date_parser(a['fields']['firstPublicationDate']),
                'title'   : a['webTitle'],
                'text'    : a['fields']['bodyText'],
                'apiURL'  : url
            })  
        except Exception as e:
            log.debug(e)
            continue

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): content.guardianapis.com
DEBUG:urllib3.connectionpool:https://content.guardianapis.com:443 "GET /search?api-key=53aee7b4-8c94-495b-b13b-ea7214ba0ca2&edition=us&section=us-news&from-date=2017-01-01&to-date=2017-02-26&page-size=200&page=1 HTTP/1.1" 429 38


KeyError: 'response'

In [9]:
r.json()

{'message': 'API rate limit exceeded'}

# Fox news

In [ ]:
# setup
# use your newsapi key
# update collection, date and source
collection = 'foxnews'
stopWords = ['video','radio','entertainment','lifestyle','lifestyle']

page = 1
earliest_date = '2017-01-01'
latest_date = '2018-07-30'
source = 'fox-news'
pageSize = 100
apiKey = 'fd1386b678fd4524b2aa84e3bee1f8f7'

params = {
    'apiKey'   : apiKey,
    'pageSize' : pageSize,
    'page'     : page,
    'from'     : earliest_date,
    'to'       : latest_date,
    'sources'  : source
}

# base url
api_url = 'https://newsapi.org/v2/everything?'

# get first request to obtain total news count
r = requests.get(api_url, params=params)
totalPages = r.json()['totalResults']//100

log.debug(f'TOTAL PAGES FOR {source}: {totalPages}')

# scrape news
for p in range(page,totalPages):
    log.debug(f'\n\n PROCESSING PAGE: {source+"?page="+str(page)}\n\n\
              ====================================\n\n')
    
    params = {
        'apiKey'   : apiKey,
        'pageSize' : pageSize,
        'page'     : page,
        'from'     : earliest_date,
        'to'       : latest_date,
        'sources'  : source
    }
    
    r = requests.get(api_url, params=params)

    page += 1
    
    for a in r.json()['articles']:
        try:
            skip = False #flag to skip scraping
            url = a['url']
            log.debug(f"Processing url: {url}")
            for stopWord in stopWords:
                if stopWord in url.lower():
                    log.debug(f'URL has {stopWord}, skipping.')
                    skip = True
            if not skip:      
                article_date = scrape(url, db, collection)
        except Exception as e:
            log.debug(e)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newsapi.org
DEBUG:urllib3.connectionpool:https://newsapi.org:443 "GET /v2/everything?apiKey=fd1386b678fd4524b2aa84e3bee1f8f7&pageSize=100&page=1&from=2017-01-01&to=2018-07-30&sources=fox-news HTTP/1.1" 200 64117
DEBUG:__main__:TOTAL PAGES FOR fox-news: 535
DEBUG:__main__:

 PROCESSING PAGE: fox-news?page=1



DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newsapi.org
DEBUG:urllib3.connectionpool:https://newsapi.org:443 "GET /v2/everything?apiKey=fd1386b678fd4524b2aa84e3bee1f8f7&pageSize=100&page=1&from=2017-01-01&to=2018-07-30&sources=fox-news HTTP/1.1" 200 64117
DEBUG:__main__:Processing url: http://video.foxnews.com/v/5815710004001/
DEBUG:__main__:URL has video, skipping.
DEBUG:__main__:Processing url: http://video.foxnews.com/v/5815709702001/
DEBUG:__main__:URL has video, skipping.
DEBUG:__main__:Processing url: http://www.foxnews.com/us/2018/07/30/trump-supporters-pelted-by-eggs-punches-can-sue-san-jos

DEBUG:__main__:URL has video, skipping.
DEBUG:__main__:Processing url: http://video.foxnews.com/v/5815658956001/
DEBUG:__main__:URL has video, skipping.
DEBUG:__main__:Processing url: http://video.foxnews.com/v/5815659998001/
DEBUG:__main__:URL has video, skipping.
DEBUG:__main__:Processing url: http://www.foxnews.com/world/2018/07/30/syrian-forces-press-offensive-despite-is-threats-to-hostages.html
DEBUG:__main__:Exctracting features from http://www.foxnews.com/world/2018/07/30/syrian-forces-press-offensive-despite-is-threats-to-hostages.html
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.foxnews.com
DEBUG:urllib3.connectionpool:http://www.foxnews.com:80 "GET /world/2018/07/30/syrian-forces-press-offensive-despite-is-threats-to-hostages.html HTTP/1.1" 200 13132
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: http://video.foxnews.com/v/5815657313001/
DEBUG:__main__:URL has video, skipping.
DEBUG:__main__:Processing url: http://video.foxnews.com/v/581565485

DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: http://www.foxnews.com/world/2018/07/30/eritrea-says-it-and-somalia-restore-diplomatic-ties.html
DEBUG:__main__:Exctracting features from http://www.foxnews.com/world/2018/07/30/eritrea-says-it-and-somalia-restore-diplomatic-ties.html
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.foxnews.com
DEBUG:urllib3.connectionpool:http://www.foxnews.com:80 "GET /world/2018/07/30/eritrea-says-it-and-somalia-restore-diplomatic-ties.html HTTP/1.1" 200 10477
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: http://www.foxnews.com/world/2018/07/30/hundreds-stranded-on-indonesian-mountain-after-earthquake.html
DEBUG:__main__:Exctracting features from http://www.foxnews.com/world/2018/07/30/hundreds-stranded-on-indonesian-mountain-after-earthquake.html
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.foxnews.com
DEBUG:urllib3.connectionpool:http://www.foxnews.com:80 "GET /world/2018/07/30/hundreds-st

DEBUG:__main__:Processing url: https://radio.foxnews.com/2018/07/30/the-fox-news-rundown-7-30-2018/
DEBUG:__main__:URL has radio, skipping.
DEBUG:__main__:Processing url: http://www.foxnews.com/politics/2018/07/30/ginsburg-to-hopes-to-serve-at-least-five-more-years-on-supreme-court.html
DEBUG:__main__:Exctracting features from http://www.foxnews.com/politics/2018/07/30/ginsburg-to-hopes-to-serve-at-least-five-more-years-on-supreme-court.html
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.foxnews.com
DEBUG:urllib3.connectionpool:http://www.foxnews.com:80 "GET /politics/2018/07/30/ginsburg-to-hopes-to-serve-at-least-five-more-years-on-supreme-court.html HTTP/1.1" 200 12835
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: http://www.foxnews.com/health/2018/07/30/ebola-scare-at-denver-hospital-after-man-just-back-from-congo-exhibited-symptoms.html
DEBUG:__main__:Exctracting features from http://www.foxnews.com/health/2018/07/30/ebola-scare-at-denver-hospital-af

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.foxnews.com
DEBUG:urllib3.connectionpool:http://www.foxnews.com:80 "GET /world/2018/07/29/turkey-leader-erdogan-warns-us-over-sanctions-threat-in-pastor-case.html HTTP/1.1" 200 13330
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: http://www.foxnews.com/world/2018/07/29/latest-half-migrants-at-center-standoff-off-malta.html
DEBUG:__main__:Exctracting features from http://www.foxnews.com/world/2018/07/29/latest-half-migrants-at-center-standoff-off-malta.html
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.foxnews.com
DEBUG:urllib3.connectionpool:http://www.foxnews.com:80 "GET /world/2018/07/29/latest-half-migrants-at-center-standoff-off-malta.html HTTP/1.1" 200 13086
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: http://www.foxnews.com/world/2018/07/29/2-us-cyclists-among-4-killed-in-hit-and-run-in-tajikistan-3-others-injured.html
DEBUG:__main__:Exctracting features from http://www

...

DEBUG:__main__:Exctracting features from http://www.foxnews.com/world/2018/06/27/watchdog-gets-authority-to-assign-blame-in-chemical-attacks.html
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.foxnews.com
DEBUG:urllib3.connectionpool:http://www.foxnews.com:80 "GET /world/2018/06/27/watchdog-gets-authority-to-assign-blame-in-chemical-attacks.html HTTP/1.1" 200 11250
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: http://www.foxnews.com/politics/2018/06/27/gop-compromise-immigration-bill-defeated-on-house-floor.html
DEBUG:__main__:Exctracting features from http://www.foxnews.com/politics/2018/06/27/gop-compromise-immigration-bill-defeated-on-house-floor.html
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.foxnews.com
DEBUG:urllib3.connectionpool:http://www.foxnews.com:80 "GET /politics/2018/06/27/gop-compromise-immigration-bill-defeated-on-house-floor.html HTTP/1.1" 200 14173
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: http:/

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.foxnews.com
DEBUG:urllib3.connectionpool:http://www.foxnews.com:80 "GET /food-drink/2018/06/27/chick-fil-worker-saves-choking-customer-with-heimlich-maneuver-hes-hero.html HTTP/1.1" 200 13264
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: http://www.foxnews.com/opinion/2018/06/27/supreme-court-s-travel-ban-decision-sends-incredible-and-clear-message-to-lower-federal-court-judges.html
DEBUG:__main__:Exctracting features from http://www.foxnews.com/opinion/2018/06/27/supreme-court-s-travel-ban-decision-sends-incredible-and-clear-message-to-lower-federal-court-judges.html
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.foxnews.com
DEBUG:urllib3.connectionpool:http://www.foxnews.com:80 "GET /opinion/2018/06/27/supreme-court-s-travel-ban-decision-sends-incredible-and-clear-message-to-lower-federal-court-judges.html HTTP/1.1" 200 14979
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: htt

DEBUG:__main__:Exctracting features from http://www.foxnews.com/opinion/2018/06/27/supreme-court-s-janus-decision-is-win-for-government-workers-and-all-americans.html
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.foxnews.com
DEBUG:urllib3.connectionpool:http://www.foxnews.com:80 "GET /opinion/2018/06/27/supreme-court-s-janus-decision-is-win-for-government-workers-and-all-americans.html HTTP/1.1" 200 14222
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: http://video.foxnews.com/v/5802570301001/
DEBUG:__main__:URL has video, skipping.
DEBUG:__main__:Processing url: http://www.foxnews.com/politics/2018/06/27/us-supreme-court-deals-blow-to-labor-unions-ruling-5-4-that-public-sector-unions-may-not-collect-mandatory-fees-from-non-members.html
DEBUG:__main__:Exctracting features from http://www.foxnews.com/politics/2018/06/27/us-supreme-court-deals-blow-to-labor-unions-ruling-5-4-that-public-sector-unions-may-not-collect-mandatory-fees-from-non-members.html
DEBU

DEBUG:urllib3.connectionpool:http://www.foxnews.com:80 "GET /world/2018/06/27/spirit-mythical-princess-looms-over-thai-cave-crisis.html HTTP/1.1" 200 14362
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: http://www.foxnews.com/world/2018/06/27/mattis-positive-on-china-talks-set-to-discuss-north-korea.html
DEBUG:__main__:Exctracting features from http://www.foxnews.com/world/2018/06/27/mattis-positive-on-china-talks-set-to-discuss-north-korea.html
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.foxnews.com
DEBUG:urllib3.connectionpool:http://www.foxnews.com:80 "GET /world/2018/06/27/mattis-positive-on-china-talks-set-to-discuss-north-korea.html HTTP/1.1" 200 12934
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: http://www.foxnews.com/us/2018/06/27/south-carolina-woman-assaulted-black-teen-at-pool-bit-deputy-on-arm-upon-arrest-police-say.html
DEBUG:__main__:Exctracting features from http://www.foxnews.com/us/2018/06/27/south-carolina-woman-assaulted

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.foxnews.com
DEBUG:urllib3.connectionpool:http://www.foxnews.com:80 "GET /science/2018/06/27/deaths-dumping-nearly-100-baby-hammerhead-sharks-being-investigated-in-hawaii.html HTTP/1.1" 200 14352
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: http://www.foxnews.com/world/2018/06/27/amnesty-intl-urges-trials-for-myanmar-military-over-rohingya.html
DEBUG:__main__:Exctracting features from http://www.foxnews.com/world/2018/06/27/amnesty-intl-urges-trials-for-myanmar-military-over-rohingya.html
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.foxnews.com
DEBUG:urllib3.connectionpool:http://www.foxnews.com:80 "GET /world/2018/06/27/amnesty-intl-urges-trials-for-myanmar-military-over-rohingya.html HTTP/1.1" 200 13919
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://radio.foxnews.com/2018/06/27/the-fox-news-rundown-6-27-2018/
DEBUG:__main__:URL has radio, skipping.
DEBUG:__main__:Pr

# NY Post

In [ ]:
collection = 'theintercept'
source = 'https://theintercept.com'



class Sel(unittest.TestCase):
    def setUp(self):
        self.driver = webdriver.Chrome()
        self.driver.implicitly_wait(30)
        self.base_url = source
        self.verificationErrors = []
        self.accept_next_alert = True
    def getPage(self):
        driver = self.driver
        delay = 2
        driver.get(self.base_url)
        html_source = driver.page_source
        self.html = html_source.encode('utf-8')
        return self.html
    def scrollDown(self):
        driver = self.driver
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        self.html = driver.page_source.encode('utf-8')
        return driver.page_source 
    def shutdown(self):
        driver = self.driver
        driver.quit()

data = Sel()

data.setUp()
data.getPage()
page = 1

utc=pytz.UTC
earliest_date = utc.localize(date_parser('2017-01-01'))

scraped_urls = []

while True:
    log.debug(f'NEXT SCROLL (#{page})')
    page += 1
    
    html = data.scrollDown()
    soup = bs.BeautifulSoup(html,'lxml')

    for link in soup.find('div', {"id":"recently"}).findAll('a'):
        url = urljoin(source, link['href'])
        if url and url not in scraped_urls:
            scraped_urls.append(url)
            log.debug(f'Processing url: {url}')
            article_date = scrape(url, db, collection)

    try:
        if article_date < earliest_date:
            log.debug(f'Reached earliest date requested: {article_date}')
            break
    except Exception as e:
        log.debug(f"Something is wrong: {e}")
data.shutdown()